<a href="https://colab.research.google.com/github/taichi0315/optimization-handson/blob/master/sample/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 数理最適化ハンズオン


## 数理最適化の概要

## 環境構築

In [1]:
!pip install pulp

     |████████████████████████████████| 40.6MB 93kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for amply: filename=amply-0.1.2-cp36-none-any.whl size=16572 sha256=3c7b5ab458f6d1bb69cfd614c0a745c155514d166d505f201c3602ad057fe516
  Stored in directory: /root/.cache/pip/wheels/84/18/f7/e5c3ed13ed5bb721763f77d4a924331d59ef115ce61c9d26eb
Successfully built amply


## ハンズオン

In [3]:
from pulp import *

### 集合
モデルを構成する要素・物体の集まり

- 職員
- シフト日付

In [66]:
# Set
Employee = { "emp1", "emp2", "emp3" }
Day       = { 1, 2, 3, 4, 5 }

In [67]:
print(Employee)

{'emp1', 'emp2', 'emp3'}


### 変数
最適化をして求めたい値。最適解が求まると変数が最適値として定まる。

- 保育士$e$が日付$d$に出勤するかどうか（0-1変数）



In [68]:
# Variable
x = {}
for emp in Employee:
    for day in Day:
        x[emp, day] = LpVariable(name=f"x[{emp}, {day}]", cat="Binary")   

In [69]:
print(x)

{('emp1', 1): x_emp1,_1_, ('emp1', 2): x_emp1,_2_, ('emp1', 3): x_emp1,_3_, ('emp1', 4): x_emp1,_4_, ('emp1', 5): x_emp1,_5_, ('emp2', 1): x_emp2,_1_, ('emp2', 2): x_emp2,_2_, ('emp2', 3): x_emp2,_3_, ('emp2', 4): x_emp2,_4_, ('emp2', 5): x_emp2,_5_, ('emp3', 1): x_emp3,_1_, ('emp3', 2): x_emp3,_2_, ('emp3', 3): x_emp3,_3_, ('emp3', 4): x_emp3,_4_, ('emp3', 5): x_emp3,_5_}


### モデルの作成

In [70]:
model = LpProblem()

### 制約
求解する上で必ず守る制約条件

- 職員の最低配置人数

In [71]:
for day in Day:
    model += (
        sum(x[emp, day] for emp in Employee)
        >=
        3
    )

### 求解

In [72]:
status = model.solve()
print(LpStatus[status])

Optimal


In [73]:
for emp,day in x:
    print(emp, day, x[emp, day].value())

emp1 1 1.0
emp1 2 1.0
emp1 3 1.0
emp1 4 1.0
emp1 5 1.0
emp2 1 1.0
emp2 2 1.0
emp2 3 1.0
emp2 4 1.0
emp2 5 1.0
emp3 1 1.0
emp3 2 1.0
emp3 3 1.0
emp3 4 1.0
emp3 5 1.0
